In [1]:
from IPython.display import Markdown, display

date = '2022-03-10'
practice = '../_practice/' + date + '.md'
prepare =  '../_prepare/' + date + '.md'

# How does ssh really work? how can I be more secure?

## Using ssh keys to authenticated

1. generate a key pair
1. store the public key on the server
1. Request to login, tell server your public key, get back a session ID from the server
1. if it has that public key, then it generates a random string, encrypts it with your public key and sends it back to your computer
1. On your computer, it decrypts the message + the session ID with your private key then hashes the message and sends it back
1. the server then hashes its copy of the message and session ID and if the hash received and calculated match, then you are loggied in



[a toy example](https://thatsmaths.com/2016/08/11/a-toy-example-of-rsa-encryption/)



![cheatsheet on ssh from julia evans](https://wizardzines.com/comics/ssh/ssh.png)

[from wizardzines](https://wizardzines.com/comics/ssh/)



Lots more networking detals in the [full zine](https://wizardzines.com/zines/bite-size-networking/) available for purchase or I have a copy if you want to borrow it.


this *free zine* describes networks at a lower level [full zine](https://jvns.ca/networking-zine.pdf) but does not include ssh


## Generating a Key Pair

We can use `ssh-keygen` to create a keys.
```
ssh-keygen -f ~/.ssh/seawulf -t rsa -b 4096
```

The `-f` option allows us to specify the file name of the keys.

```
Generating public/private rsa key pair.
Enter passphrase (empty for no passphrase):
Enter same passphrase again:
Your identification has been saved in /Users/brownsarahm/.ssh/seawulf.
Your public key has been saved in /Users/brownsarahm/.ssh/seawulf.pub.
The key fingerprint is:
SHA256:e1g1ZJA3nlIvMs0mpFG8Sca9z9DQl02N14/Z6Y9Xj70 brownsarahm@152.133.20.172.s.wireless.uri.edu
The key's randomart image is:
+---[RSA 4096]----+
|         +o+o. o*|
|        . Bo* o.*|
|         * OoB *o|
|        . B.X.=.o|
|        S .* =.  |
|         +    o..|
|        o .    ++|
|         .    o =|
|               E.|
+----[SHA256]-----+
```


## Sending the public key to a server


```
ssh-copy-id -i ~/.ssh/seawulf brownsarahm@seawulf.uri.edu
```

and we see it working :
```
/usr/bin/ssh-copy-id: INFO: Source of key(s) to be installed: "/Users/brownsarahm/.ssh/seawulf.pub"
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed
/usr/bin/ssh-copy-id: INFO: 1 key(s) remain to be installed -- if you are prompted now it is to install the new keys
```
Then you have to authenticate with your password, because that is what was
set up before.

```
Number of key(s) added:        1

Now try logging into the machine, with:   "ssh 'brownsarahm@seawulf.uri.edu'"
and check to make sure that only the key(s) you wanted were added.
```


To login without usng a password you have to tell ssh which key to use:
```
ssh -i ~/.ssh/seawulf brownsarahm@seawulf.uri.edu
```

Or you can add the following to your `~/.ssh/config` file
```
Host seawulf
  Hostname seawulf.uri.edu
  Username brownsarahm
  IdentityFile ~/.ssh/seawulf
```

and then use
```
ssh seawulf
```


## Review: Undoing half a commit

We can undo a commit using revert, but that applies the opposite of all changes made with that commit.  What can we do to undo one of the changes, but not the others?


We need a new commit that has only the changes we actually want to revert.  


We can checkout the new commit and then reset the head. Then we have the changes in question in the staging area ready for commit.  Now we can commit only the one we want to do in a new commit.  


Then we can checkout the main branch, and use git revert to appy the opposite of the thing we want to undo.  

## Review: Grep, head, and tail to keep excerpts of a file

```
git add .
git commit -m 'export'
```

```
[nots38 3831319] export
g 1 file changed, 579569 insertions(+)
 create mode 100644 notes/2022-03-08.md
```

When I pushed it though, is where I realized my problem:
```
$ git push --set-upstream origin nots38
```

It looked normal at first
```
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 4.43 MiB | 4.17 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
remote: warning: See http://git.io/iEPt8g for more information.
remote: warning: File notes/2022-03-08.md is 78.68 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.
remote:
remote: Create a pull request for 'nots38' on GitHub by visiting:
remote:      https://github.com/introcompsys/spring2022/pull/new/nots38
remote:
To https://github.com/introcompsys/spring2022.git
 * [new branch]      nots38 -> nots38
Branch 'nots38' set up to track remote branch 'nots38' from 'origin'.
```

but the message reminds me that I added a very large file and it took longer than normal.  That file is 78.68 MB and GitHub prefers that you not track files larger than 50MB.


However, I knew that the large output was caused by a specific command, so I used `grep` to find what line of the file that occured.
```
grep -n "cat dmel-all-r6.19.gtf" notes/2022-03-08.md
```

```
76:[brownsarahm@seawulf ~]$ cat dmel-all-r6.19.gtf
```

So then I used `head` to take the lines above that plus one below it into a new file

```
head -n 77 notes/2022-03-08.md >> notes/2022-03-08-1.md
```

Then I found the first time we used `head` in class because I knew that was next

```
grep -n "head" notes/2022-03-08.md
```


```
542125:[brownsarahm@seawulf ~]$ head dmel-all-r6.19.gtf
542194:[brownsarahm@seawulf ~]$ head dmel-all-r6.19.gtf
```

ANd used tail to write the file from that line to the end to a new file.

```
tail -n +542124 notes/2022-03-08.md >> notes/2022-03-08-tail.md
```

the `-n` parameter allows you to say how manny lines, but if you put a + before the number it starts *from* that line to end instead of specifying how many lines.

Then I knew that the next long thing was when we used grep on mRNA, so I found that line:

```
grep -n "grep" notes/2022-03-08-tail.md
```
Since I didn't use mrna in the grep, I got three results:

```
70:[brownsarahm@seawulf ~]$ grep Act5c dmel-all-r6.19.gtf
82:[brownsarahm@seawulf ~]$ grep mRNA dmel-all-r6.19.gtf
34108:[brownsarahm@seawulf ~]$ grep mRNA dmel-all-r6.19.gtf | wc -l
```

And again I cna take the head of the file and write it to a new file.
```
head -n 83 notes/2022-03-08-tail.md > notes/2022-03-08-2.md
```

From that same last grep I also knew what the new end of file was:
```
tail -n +34107 notes/2022-03-08-tail.md > notes/2022-03-08-t2.md
```

I thought this might be done, but I checked:

```
wc -l notes/2022-03-08-t2.md
    3340 notes/2022-03-08-t2.md
```

that was still a lot of lines, and then I remembered we had a lot of output from `lshw`

```
grep -n lshw notes/2022-03-08-t2.md
```

which gave one result
```
152:[I have no name!@n005 ~]$ lshw
```

that told me the third section f the file I wanted:

```
head -n 153 notes/2022-03-08-t2.md > notes/2022-03-08-3.md
```

And I knew after `lshw` we exited the compute node:

```
grep -n exit notes/2022-03-08-t2.md
```

```
3323:[I have no name!@n005 ~]$ exit
3337:[brownsarahm@seawulf ~]$ exit
```

Which allowed me to create the fourth section that I needed:

```
tail -n +3321 notes/2022-03-08-t2.md > notes/2022-03-08-4.md
```

Next I removed the original file from git without deleting it locally
```
git reset --mixed HEAD~1
git status
```

git `rm` did not work because that left the file hashed into the git directory and I wanted to fully remove it. SO I reset the head back on setup, to undo the last commit. `--mixed` makes it delete from the repo for real, but not delete locally.

Now I see that it doesn't know anything about that file.
```
On branch nots38
Your branch is behind 'origin/nots38' by 1 commit, and can be fast-forwarded.
  (use "git pull" to update your local branch)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	notes/2022-03-08-1.md
	notes/2022-03-08-2.md
	notes/2022-03-08-3.md
	notes/2022-03-08-4.md
	notes/2022-03-08-t2.md
	notes/2022-03-08-tail.md
	notes/2022-03-08.md
	notes/2022-03-08cmdonly.md
```


Then I renamed the original file:
```
mv notes/2022-03-08.md notes/2022-03-08-all.md
```

I tried combinging them with echo:

```
echo notes/2022-03-08-1.md >> notes/2022-03-08.md
echo notes/2022-03-08-2.md >> notes/2022-03-08.md
echo notes/2022-03-08-3.md >> notes/2022-03-08.md
echo notes/2022-03-08-4.md >> notes/2022-03-08.md
```

And confirmed the number of lines in the combined file
```
wc -l notes/2022-03-08.md
```

Which is how I learned that did not works
```
       4 notes/2022-03-08.md
```


Then I remembered to use `cat`

```
cat notes/2022-03-08-1.md >> notes/2022-03-08.md
cat notes/2022-03-08-2.md >> notes/2022-03-08.md
cat notes/2022-03-08-3.md >> notes/2022-03-08.md
cat notes/2022-03-08-4.md >> notes/2022-03-08.md
```

Again checking the number of lines to show that it worked:
```
wc -l notes/2022-03-08.md
```

```
     333 notes/2022-03-08.md
```

and for rference comapred to the original
```
wc -l notes/2022-03-08-all.md
```
and we see the original length
```
  579569 notes/2022-03-08-all.md
```

Then I looked at the compiled file and deleted the others once I was sure I had what I needed.

(class15prep)=
## Prepare for next class
````{margin}
```{note}
The text in () in these sections is an explanation of *why* that task is assigned
```
````

In [2]:
display(Markdown(prepare))


1. Do what you need to do to finish the semester well: eg rest, catchup on work, both (breaks are important)
1. Review your IDE thoughts right before class on Tuesday March 22 (so you're ready for the activity in class)


(class15practice)=
## More Practice

In [3]:
display(Markdown(practice))

1. **optional** Configure ssh keys to your [GitHub account](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/adding-a-new-ssh-key-to-your-github-account) (this is actually GitHub's preferred terminal authentication method)


## Questions after class